# Members Names
1. Muhammad Hassan Saboor (23L-8006)
2. Muhammad Haider Abbas  (23L-8017)
3. Muhammad Farhan Naveed (23L-8024)

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign

gtsrb-german-traffic-sign.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/gtsrb-german-traffic-sign.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
X = []
Y = []
total_class = 43
cur_directory = os.getcwd()

In [ ]:
for index in range(total_class):
    path = os.path.join(cur_directory, 'train', str(index))
    images = os.listdir(path)

    # Iterating on all the images of the index folder
    for img in images:
        try:
            image = Image.open(os.path.join(path, img))
            image = image.resize((30, 30))
            image = np.array(image)
            X.append(image)
            Y.append(index)
        except:
            print("Error loading image")

X = np.array(X)
Y = np.array(Y)

print(X.shape, Y.shape)

(39209, 30, 30, 3) (39209,)


In [ ]:
# Splitting the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

print("Shape of x_train:", x_train.shape, "and y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape, "and y_test:", y_test.shape)

# One hot encoding the labels
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

Shape of x_train: (31367, 30, 30, 3) and y_train: (31367,)
Shape of x_test: (7842, 30, 30, 3) and y_test: (7842,)


In [ ]:
# Existing layers
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=x_train.shape[1:]))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

# Additional layer
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))  # Additional Convolutional layer
model.add(MaxPool2D(pool_size=(2, 2)))  # Additional MaxPooling layer
model.add(Dropout(rate=0.25))  # Additional Dropout layer

# Flatten layer and fully connected layers
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print("Shapes - x_train:", x_train.shape, "y_train:", y_train.shape)
print("Shapes - x_test:", x_test.shape, "y_test:", y_test.shape)

Shapes - x_train: (31367, 30, 30, 3) y_train: (31367, 43)
Shapes - x_test: (7842, 30, 30, 3) y_test: (7842, 43)


In [ ]:
epochs = 25
history = model.fit(x_train, y_train, batch_size=64, epochs=epochs, validation_data=(x_test, y_test))
model.save('traffic_recognition.h5')

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from PIL import Image
import numpy as np

y_test = pd.read_csv('Test.csv')
labels = y_test["ClassId"].values
img_paths = y_test["Path"].values

test_data = []
for path in img_paths:
    image = Image.open(path)
    image = image.resize((30, 30))
    test_data.append(np.array(image))

test_data = np.array(test_data)

pred_probabilities = model.predict(test_data)
pred = np.argmax(pred_probabilities, axis=1)



print("Classification Report:")
print(classification_report(labels, pred))


In [ ]:
accuracy = accuracy_score(labels, pred)
print("Accuracy with the test data:", accuracy*100)

In [ ]:
# Generate classification report and store its values
classification_rep = classification_report(labels, pred, output_dict=True)
precision = [classification_rep[str(i)]['precision'] for i in range(len(classification_rep) - 3)]
recall = [classification_rep[str(i)]['recall'] for i in range(len(classification_rep) - 3)]
f1_score = [classification_rep[str(i)]['f1-score'] for i in range(len(classification_rep) - 3)]
class_names = [str(i) for i in range(len(classification_rep) - 3)]

In [ ]:
# Plotting the metrics
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# Precision plot
ax[0].bar(class_names, precision, color='skyblue')
ax[0].set_xlabel('Precision')
ax[0].set_title('Precision per Class')

# Recall plot
ax[1].bar(class_names, recall, color='lightgreen')
ax[1].set_xlabel('Recall')
ax[1].set_title('Recall per Class')

# F1-score plot
ax[2].bar(class_names, f1_score, color='salmon')
ax[2].set_xlabel('F1-score')
ax[2].set_title('F1-score per Class')

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# Precision plot (Line Chart)
ax[0].plot(class_names, precision, marker='o', linestyle='-', color='skyblue')
ax[0].set_xlabel('Classes')
ax[0].set_ylabel('Precision')
ax[0].set_title('Precision per Class')
ax[0].grid(True)

# Recall plot (Line Chart)
ax[1].plot(class_names, recall, marker='o', linestyle='-', color='lightgreen')
ax[1].set_xlabel('Classes')
ax[1].set_ylabel('Recall')
ax[1].set_title('Recall per Class')
ax[1].grid(True)

# F1-score plot (Line Chart)
ax[2].plot(class_names, f1_score, marker='o', linestyle='-', color='salmon')
ax[2].set_xlabel('Classes')
ax[2].set_ylabel('F1-score')
ax[2].set_title('F1-score per Class')
ax[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 5))

# Plotting training and validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
from PIL import Image
import numpy as np
from keras.models import load_model

# Load the trained model
model = load_model('traffic_recognition.h5')

# Dictionary to label all traffic signs classes
classes = {
    1: 'Speed limit (20km/h)',
    2: 'Speed limit (30km/h)',
    3: 'Speed limit (50km/h)',
    4: 'Speed limit (60km/h)',
    5: 'Speed limit (70km/h)',
    6: 'Speed limit (80km/h)',
    7: 'End of speed limit (80km/h)',
    8: 'Speed limit (100km/h)',
    9: 'Speed limit (120km/h)',
    10: 'No passing',
    11: 'No passing veh over 3.5 tons',
    12: 'Right-of-way at intersection',
    13: 'Priority road',
    14: 'Yield',
    15: 'Stop',
    16: 'No vehicles',
    17: 'Veh > 3.5 tons prohibited',
    18: 'No entry',
    19: 'General caution',
    20: 'Dangerous curve left',
    21: 'Dangerous curve right',
    22: 'Double curve',
    23: 'Bumpy road',
    24: 'Slippery road',
    25: 'Road narrows on the right',
    26: 'Road work',
    27: 'Traffic signals',
    28: 'Pedestrians',
    29: 'Children crossing',
    30: 'Bicycles crossing',
    31: 'Beware of ice/snow',
    32: 'Wild animals crossing',
    33: 'End speed + passing limits',
    34: 'Turn right ahead',
    35: 'Turn left ahead',
    36: 'Ahead only',
    37: 'Go straight or right',
    38: 'Go straight or left',
    39: 'Keep right',
    40: 'Keep left',
    41: 'Roundabout mandatory',
    42: 'End of no passing',
    43: 'End no passing veh > 3.5 tons'
}


def predict_traffic_sign(image_path):
    image = Image.open(image_path)
    image = image.resize((30, 30))
    image = np.array(image)
    image = np.expand_dims(image, axis=0)

    pred_probabilities = model.predict(image)
    pred_class = np.argmax(pred_probabilities, axis=1)[0]
    sign = classes[pred_class + 1]

    return sign

# Example usage:
image_path_to_test = 'image url'  # Replace this with the path to your test image
predicted_sign = predict_traffic_sign(image_path_to_test)
print("Predicted Traffic Sign:", predicted_sign)